# AED exploration and opening hours

In [5]:
import openrouteservice
from openrouteservice import client
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import geometry
import time
import geopy.distance

import pandas as pd
import numpy as np
import openrouteservice
from openrouteservice import client
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import geometry
import time
import geopy.distance


import folium
import geopandas as gpd
from shapely.geometry import Point
import random

In [6]:
AEDs = gpd.read_file('C:/Users/leonw/OneDrive - KU Leuven/Documents/GitHub/AED-Route-Optimization-MDA-Project/Data/temp.gpkg', layer='AED_data')

In [7]:
AEDs.head()

,lon,lat,geometry
0,4.737563,50.847642,"POLYGON ((4.72665 50.84916, 4.72657 50.84817, ..."
1,4.696771,50.882662,"POLYGON ((4.68680 50.88487, 4.68743 50.88203, ..."
2,4.705893,50.891915,"POLYGON ((4.69766 50.88972, 4.69971 50.88766, ..."
3,4.718684,50.893646,"POLYGON ((4.71665 50.89818, 4.71672 50.89643, ..."
4,4.693191,50.869499,"POLYGON ((4.68327 50.86743, 4.68455 50.86551, ..."


In [9]:
import folium
import geopandas as gpd
from shapely.geometry import Point
import openrouteservice

def find_isochrone(coordinate, profile="foot-walking", threshold=600):
    isochrones_parameters = {
        'locations': coordinate,
        'profile': profile,
        'range_type': 'time',
        'range': [threshold]  # 10 minutes (600 seconds)
    }

    ors_client_custom = openrouteservice.Client(key='5b3ce3597851110001cf624802e069d6633748a5ae4e9842334f1dc2')
    isochrone = ors_client_custom.isochrones(**isochrones_parameters)
    
    return isochrone

# Define the patient's location
coordinate = [[4.69932, 50.88762]]
patients = (4.69932, 50.88762)
# Fetch the isochrone for the patient
iso_patient = find_isochrone(coordinate)



In [10]:
gdf = AEDs[0:3].copy()
gdf.loc[:,'AED'] = list(zip(gdf['lon'], gdf['lat']))
gdf.head()


,lon,lat,geometry,AED
0,4.737563,50.847642,"POLYGON ((4.72665 50.84916, 4.72657 50.84817, ...","(4.737563, 50.8476423)"
1,4.696771,50.882662,"POLYGON ((4.68680 50.88487, 4.68743 50.88203, ...","(4.6967705, 50.8826624)"
2,4.705893,50.891915,"POLYGON ((4.69766 50.88972, 4.69971 50.88766, ...","(4.7058934, 50.89191539999999)"


In [11]:

# Convert AED column to Points if not already in geometry format
if not isinstance(gdf['AED'].iloc[0], Point):
    gdf['AED'] = gdf['AED'].apply(lambda coords: Point(coords) if isinstance(coords, (list, tuple)) else coords)

# Set AED as the active geometry column in the GeoDataFrame
gdf = gdf.set_geometry('AED')

In [12]:
import folium
import geopandas as gpd
import random
from shapely.geometry import Point
import openrouteservice
import numpy as np
def generate_random_points(center, num_points=10, radius=500):
    """Generate random points around a given center within a radius in meters."""
    lat, lon = center
    earth_radius = 6371000  # meters

    points = []
    for _ in range(num_points):
        # Generate random distances within the radius
        rand_dist = random.uniform(0, radius)
        rand_angle = random.uniform(0, 2 * np.pi)

        # Convert meters to degrees (approximation)
        delta_lat = (rand_dist / earth_radius) * (180 / np.pi)
        delta_lon = (rand_dist / (earth_radius * np.cos(np.radians(lat)))) * (180 / np.pi)

        new_lat = lat + delta_lat * np.cos(rand_angle)
        new_lon = lon + delta_lon * np.sin(rand_angle)

        points.append((new_lat, new_lon))

    return points

In [13]:
# Generate random points around the patient
random_points = generate_random_points(patients, num_points=10, radius=1000)
# Convert random points to GeoDataFrame
random_gdf = gpd.GeoDataFrame(geometry=[Point(lon, lat) for lat, lon in random_points])

In [ ]:
# Randomly generate numbers with a decline_rate. 
# Data frame with all the possible responders and their probability to accept the call. 1 for acceptance
acceptance = list(random.choice([0,1], p=[decline_rate, (1-decline_rate)], size = len(possible_responder)))
df_acceptance = pd.DataFrame({'Responder_loc' : possible_responder,
                                'Probability' : acceptance})
        

In [14]:
# Initialize a folium map centered around the patient’s location
m = folium.Map(location=[patients[1], patients[0]], zoom_start=13)

# Add the patient's isochrone (in red)
folium.GeoJson(
    iso_patient,
    style_function=lambda x: {
        'fillColor': 'red',
        'color': 'red',
        'weight': 2,
        'fillOpacity': 0.3
    }
).add_to(m)

# Add the patient marker (in green)
folium.Marker(
    location=[patients[1], patients[0]],
    popup="Patient",
    icon=folium.Icon(color='green', icon='user', prefix='fa')
).add_to(m)

m

In [16]:

# Initialize a folium map centered around the patient’s location
m = folium.Map(location=[patients[1], patients[0]], zoom_start=15)

# Add the patient's isochrone (in red)
folium.GeoJson(
    iso_patient,
    style_function=lambda x: {
        'fillColor': 'red',
        'color': 'red',
        'weight': 2,
        'fillOpacity': 0.3
    }
).add_to(m)

# Add patient marker (green)
folium.Marker(
    location=[patients[1], patients[0]],
    popup="Patient",
    icon=folium.Icon(color='red', icon='user', prefix='fa')
).add_to(m)


# Add random points (blue)
for lat, lon in random_points:
    folium.Marker(
        location=[lon, lat],
        popup="Random Point",
        icon=folium.Icon(color='blue', icon='user', prefix='fa')
    ).add_to(m)

m

In [15]:
# Initialize a folium map centered around the patient’s location
m = folium.Map(location=[patients[1], patients[0]], zoom_start=15)

# Add the patient's isochrone (in red)
folium.GeoJson(
    iso_patient,
    style_function=lambda x: {
        'fillColor': 'red',
        'color': 'red',
        'weight': 2,
        'fillOpacity': 0.3
    }
).add_to(m)

# Add patient marker (green)
folium.Marker(
    location=[patients[1], patients[0]],
    popup="Patient",
    icon=folium.Icon(color='red', icon='user', prefix='fa')
).add_to(m)

# Add AEDs markers (red)
for _, row in gdf.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup="AED",
        icon=folium.Icon(color='green', icon='heartbeat', prefix='fa')
    ).add_to(m)

# Add random points (blue)
for lat, lon in random_points:
    folium.Marker(
        location=[lon, lat],
        popup="Random Point",
        icon=folium.Icon(color='blue', icon='user', prefix='fa')
    ).add_to(m)

# Show the map
m

In [17]:
# Initialize a folium map centered around the patient’s location
m = folium.Map(location=[patients[1], patients[0]], zoom_start=13)

# Add the patient's isochrone (in red)
folium.GeoJson(
    iso_patient,
    style_function=lambda x: {
        'fillColor': 'red',
        'color': 'red',
        'weight': 2,
        'fillOpacity': 0.3
    }
).add_to(m)

# Add the patient marker (in green)
folium.Marker(
    location=[patients[1], patients[0]],
    popup="Patient",
    icon=folium.Icon(color='green', icon='user', prefix='fa')
).add_to(m)

# Add isochrones for AEDs (in blue)
for _, row in gdf.iterrows():
    folium.GeoJson(
        row['geometry'],
        style_function=lambda x: {
            'fillColor': 'blue',
            'color': 'blue',
            'weight': 2,
            'fillOpacity': 0.3
        }
    ).add_to(m)

# Add AED markers (in red)
for _, row in gdf.iterrows():
    folium.Marker(
        location=[row['AED'].y, row['AED'].x],
        popup="AED",
        icon=folium.Icon(color='red', icon='heartbeat', prefix='fa')
    ).add_to(m)

# Add random points (blue)
for lat, lon in random_points:
    folium.Marker(
        location=[lon, lat],
        popup="Random Point",
        icon=folium.Icon(color='blue', icon='user', prefix='fa')
    ).add_to(m)

# Show the map

m

In [25]:
# Filter points inside the isochrone polygon
# Convert Isochrone to Polygon
from shapely.geometry import Point, Polygon, shape
iso_polygon = shape(iso_patient['features'][0]['geometry'])



In [30]:
gdf_iso_polygon = gpd.GeoDataFrame({'geometry': [geometry.Polygon(iso_patient['features'][0]['geometry']['coordinates'][0])]})
gdf_iso_polygon

,geometry
0,"POLYGON ((4.68929 50.88522, 4.69125 50.88321, ..."


In [31]:
random_points

[(4.699494020757639, 50.89538254337843),
 (4.696070395010916, 50.895973989960524),
 (4.6988937307183765, 50.88928789041148),
 (4.694073171314823, 50.88456620138093),
 (4.69423913260052, 50.89444381614162),
 (4.697383285811491, 50.88792467151592),
 (4.6983404708145855, 50.891136735680725),
 (4.699814863109981, 50.88963071924299),
 (4.699326605146951, 50.88753526476586),
 (4.706083456934369, 50.88740452107823)]

In [42]:

filter_points = list([geometry.Point(random_points[i][0],random_points[i][1]) for i in range(len(random_points))])
filter_points


[<POINT (4.699 50.895)>,
 <POINT (4.696 50.896)>,
 <POINT (4.699 50.889)>,
 <POINT (4.694 50.885)>,
 <POINT (4.694 50.894)>,
 <POINT (4.697 50.888)>,
 <POINT (4.698 50.891)>,
 <POINT (4.7 50.89)>,
 <POINT (4.699 50.888)>,
 <POINT (4.706 50.887)>]

In [44]:
filter_points.y

AttributeError: 'list' object has no attribute 'y'

In [45]:
coordinate_tuples = [[(point)] for point in filter_points if point.within(gdf_iso_polygon.loc[0,'geometry'])]
df_filter_points = pd.DataFrame(coordinate_tuples, columns = ['coordinates'])

In [48]:
df_filter_points.loc[1]['coordinates'].y

50.88456620138093

In [53]:
# Initialize a folium map centered around the patient’s location
m = folium.Map(location=[patients[1], patients[0]], zoom_start=13)

# Add the patient's isochrone (in red)
folium.GeoJson(
    iso_patient,
    style_function=lambda x: {
        'fillColor': 'red',
        'color': 'red',
        'weight': 2,
        'fillOpacity': 0.15
    }
).add_to(m)

# Add the patient marker (in green)
folium.Marker(
    location=[patients[1], patients[0]],
    popup="Patient",
    icon=folium.Icon(color='red', icon='user', prefix='fa')
).add_to(m)

# Add isochrones for AEDs (in blue)
for _, row in gdf.iterrows():
    folium.GeoJson(
        row['geometry'],
        style_function=lambda x: {
            'fillColor': 'green',
            'color': 'green',
            'weight': 2,
            'fillOpacity': 0.15
        }
    ).add_to(m)

# Add AED markers (in red)
for _, row in gdf.iterrows():
    folium.Marker(
        location=[row['AED'].y, row['AED'].x],
        popup="AED",
        icon=folium.Icon(color='green', icon='heartbeat', prefix='fa')
    ).add_to(m)

# Add random points (blue)
# Add filtered random points (Blue)
for _,row in df_filter_points.iterrows():
    folium.Marker(
        location=[row['coordinates'].y, row['coordinates'].x],
        popup="Valid Point",
        icon=folium.Icon(color='blue', icon='user', prefix='fa')
    ).add_to(m)
# Show the map

m

# Now route

In [54]:

# Initialize ORS Client
ORS_API_KEY = '5b3ce3597851110001cf624802e069d6633748a5ae4e9842334f1dc2'
ors_client = openrouteservice.Client(key=ORS_API_KEY)

In [56]:
def directions(client, coordinates, profile='foot-walking', sleep=1.5):
    time.sleep(sleep)  # To avoid API rate limits
    route = client.directions(
        coordinates=coordinates,
        profile=profile,
        format='geojson',
        validate=False
    )

    return {
        'duration': route.get('features')[0]['properties']['summary']['duration'],
        'coordinates': route.get('features')[0]['geometry']['coordinates']
    }

In [ ]:
# Compute routes from each valid point to the patient
routes = []
for point in df_filter_points:
    route = directions(ors_client, [(point.y, pointx), (patients[1], patients[0])])
    routes.append(route)

In [58]:
routes

[]

In [ ]:

# Initialize a folium map
m = folium.Map(location=[patients[0], patients[1]], zoom_start=15)

# Add patient marker (Green)
folium.Marker(
    location=[patients[0], patients[1]],
    popup="Patient",
    icon=folium.Icon(color='green', icon='user', prefix='fa')
).add_to(m)

# Add AED markers (Red)
for _, row in AEDs.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup="AED",
        icon=folium.Icon(color='red', icon='heartbeat', prefix='fa')
    ).add_to(m)

# Add filtered random points (Blue)
for lat, lon in filtered_points:
    folium.Marker(
        location=[lat, lon],
        popup="Valid Point",
        icon=folium.Icon(color='blue', icon='cloud', prefix='fa')
    ).add_to(m)

# Add routes (Yellow)
for route in routes:
    folium.PolyLine(
        locations=[(coord[1], coord[0]) for coord in route['coordinates']],
        color="yellow",
        weight=2.5,
        opacity=1
    ).add_to(m)